# Football Match Prediction - Clean Model Building

Saubere Implementation mit RandomForest und umfassendem Hyperparameter-Tuning.


In [41]:
import pickle
import pandas as pd
import numpy as np

# Load data
match_df = pickle.load(open("data/prepped_match_df.pck", "rb"))
print(f"Dataset loaded: {match_df.shape}")
match_df.head()


Dataset loaded: (4832, 67)


,teamHomeId,teamAwayId,matchDay,season,teamHomeValue,teamAwayValue,teamHomeAvgScoredGoals1,teamHomeAvgGottenGoals1,teamHomeAvgTeamPoints1,teamHomeAvgScoredGoals2,...,teamAwayAvgScoredGoals8,teamAwayAvgGottenGoals8,teamAwayAvgTeamPoints8,teamAwayAvgScoredGoals9,teamAwayAvgGottenGoals9,teamAwayAvgTeamPoints9,teamAwayAvgScoredGoals10,teamAwayAvgGottenGoals10,teamAwayAvgTeamPoints10,resultClass
90,7,54,11,2009,109.40,61.98,1.0,1.0,1.0,1.5,...,0.625,2.625,0.125,0.666667,2.555556,0.111111,0.7,2.3,0.4,2:0
93,100,87,11,2009,136.28,55.00,3.0,3.0,1.0,1.5,...,0.625,1.750,0.500,0.777778,1.666667,0.777778,1.0,1.8,0.8,2:3
91,131,81,11,2009,155.43,48.78,0.0,0.0,1.0,1.0,...,1.500,1.375,1.875,1.444444,1.333333,1.777778,1.5,1.4,1.7,3:3
92,16,40,11,2009,124.65,279.55,0.0,1.0,0.0,0.5,...,1.875,0.875,2.000,1.777778,0.888889,1.888889,1.7,0.9,1.8,0:0
94,65,55,11,2009,59.13,50.88,0.0,0.0,1.0,0.5,...,1.500,1.250,1.375,1.444444,1.222222,1.333333,1.3,1.2,1.2,0:1


In [42]:
# Prepare target variable
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
match_df["resultClass"] = enc.fit_transform(match_df["resultClass"])


In [43]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    match_df.drop("resultClass", axis=1), 
    match_df["resultClass"], 
    test_size=0.2, 
    shuffle=False  # Time series data
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


Training set: (3865, 66)
Test set: (967, 66)


In [44]:
# Custom Kicktipp Scorer
from sklearn.metrics import make_scorer

def kicktipp_score(y_true, y_pred, **kwargs):
    """Custom scoring function for Kicktipp predictions"""
    # Convert to int64 for consistency
    y_true = y_true.astype('int64') if hasattr(y_true, 'astype') else np.array(y_true, dtype='int64')
    y_pred = y_pred.astype('int64') if hasattr(y_pred, 'astype') else np.array(y_pred, dtype='int64')
    
    # Decode predictions back to score format
    y_true_decoded = [x.split(":") for x in enc.inverse_transform(y_true)]
    y_pred_decoded = [x.split(":") for x in enc.inverse_transform(y_pred)]
    
    score_value = 0
    for true, pred in zip(y_true_decoded, y_pred_decoded):
        # Exact match: 5 points
        if true[0] == pred[0] and true[1] == pred[1]:
            score_value += 5
        # Goal difference correct: 3 points
        elif (int(true[0]) - int(true[1])) == (int(pred[0]) - int(pred[1])):
            score_value += 3
        # Winner correct: 1 point
        elif ((true[0] > true[1]) and (pred[0] > pred[1])) or \
             ((true[0] < true[1]) and (pred[0] < pred[1])) or \
             ((true[0] == true[1]) and (pred[0] == pred[1])):
            score_value += 1
    
    return round(score_value / (len(y_true)/306))  # Normalize to Kicktipp scale

kicktipp_scorer = make_scorer(kicktipp_score, greater_is_better=True)
print("✅ Custom Kicktipp scorer ready")

# Cross-Validation Setup
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
print("✅ Time Series Cross-Validation ready (5 splits)")

✅ Custom Kicktipp scorer ready
✅ Time Series Cross-Validation ready (5 splits)


In [45]:
# Baseline Model
from collections import Counter

# Always predict most frequent class
majority_class = Counter(y_train).most_common(1)[0][0]
baseline_predictions = np.full(len(y_test), majority_class)
baseline_score = kicktipp_score(y_test, baseline_predictions)

print(f"📊 Baseline (majority class): {baseline_score} points")


📊 Baseline (majority class): 301 points


# Random Forest

In [46]:
# RandomForest with Comprehensive Hyperparameter Tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

print("🌳 Setting up RandomForest with comprehensive hyperparameter search...")

# Comprehensive parameter grid
param_distributions = {
    'n_estimators': np.arange(100, 501, 25),           # 100, 125, 150, ..., 500
    'max_depth': [None] + list(np.arange(5, 31, 2)),   # None, 5, 7, 9, ..., 29
    'min_samples_split': np.arange(2, 21, 1),          # 2, 3, 4, ..., 20
    'min_samples_leaf': np.arange(1, 16, 1),           # 1, 2, 3, ..., 15
    'max_features': ['sqrt', 'log2', None] + list(np.arange(0.1, 1.0, 0.1)),  # sqrt, log2, None, 0.1-0.9
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes': [None] + list(np.arange(10, 101, 10))  # None, 10, 20, ..., 100
}

# Calculate total combinations
total_combinations = 1
for param, values in param_distributions.items():
    total_combinations *= len(values)

print(f"📊 Hyperparameter space: {total_combinations:,} total combinations")
print(f"📊 Search iterations: 150 ({150/total_combinations*100:.4f}% coverage)")

# RandomForest model
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# RandomizedSearchCV
rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=150,  # Comprehensive search
    cv=tscv,
    scoring=kicktipp_scorer,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("🚀 Starting hyperparameter optimization...")
rf_search.fit(X_train, y_train)

# Results
best_params = rf_search.best_params_
best_cv_score = rf_search.best_score_
best_model = rf_search.best_estimator_

print(f"\n🎯 Best Parameters: {best_params}")
print(f"🎯 Best CV Score: {best_cv_score:.1f} points")


🌳 Setting up RandomForest with comprehensive hyperparameter search...
📊 Hyperparameter space: 35,814,240 total combinations
📊 Search iterations: 150 (0.0004% coverage)
🚀 Starting hyperparameter optimization...
Fitting 5 folds for each of 150 candidates, totalling 750 fits


/Users/maxlautenbach/PycharmProjects/football_match_prediction/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



🎯 Best Parameters: {'n_estimators': np.int64(325), 'min_samples_split': np.int64(6), 'min_samples_leaf': np.int64(2), 'max_leaf_nodes': np.int64(10), 'max_features': None, 'max_depth': np.int64(11), 'criterion': 'gini', 'bootstrap': True}
🎯 Best CV Score: 343.4 points


In [49]:
# Final Model Evaluation
predictions = best_model.predict(X_test)
final_score = kicktipp_score(y_test, predictions)

print("=" * 50)
print("🏆 FINAL RESULTS")
print("=" * 50)
print(f"📊 Baseline Score:        {baseline_score} points")
print(f"🌳 RandomForest CV Score: {best_cv_score:.1f} points")
print(f"🌳 RandomForest Test Score: {final_score} points")
print(f"📈 Improvement over baseline: +{final_score - baseline_score} points")
print("=" * 50)

if final_score > baseline_score:
    print(f"✅ SUCCESS: {final_score - baseline_score} points improvement!")
else:
    print(f"❌ Model performs worse than baseline by {baseline_score - final_score} points")


🏆 FINAL RESULTS
📊 Baseline Score:        301 points
🌳 RandomForest CV Score: 343.4 points
🌳 RandomForest Test Score: 326 points
📈 Improvement over baseline: +25 points
✅ SUCCESS: 25 points improvement!


In [48]:
# Save Model and Encoder
from datetime import datetime

# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save best model
model_filename = f"classifier_{timestamp}.pck"
pickle.dump(best_model, open(model_filename, "wb"))

# Save encoder (fixed filename)
encoder_filename = "encoder.pck"
pickle.dump(enc, open(encoder_filename, "wb"))

print(f"💾 Model saved as: {model_filename}")
print(f"💾 Encoder saved as: {encoder_filename}")
print(f"💾 Final score: {final_score} points")
print(f"💾 Best parameters: {best_params}")


💾 Model saved as: classifier_20250928_125632.pck
💾 Encoder saved as: encoder.pck
💾 Final score: 326 points
💾 Best parameters: {'n_estimators': np.int64(325), 'min_samples_split': np.int64(6), 'min_samples_leaf': np.int64(2), 'max_leaf_nodes': np.int64(10), 'max_features': None, 'max_depth': np.int64(11), 'criterion': 'gini', 'bootstrap': True}


In [52]:
# Stacking Ensemble: Top 3 RandomForest Models (corrected)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd

print("🏗️ Building Stacking Ensemble from Top 3 RandomForest models...")

# Extrahiere Top 3 Modelle aus der rf_search (nicht pipeline_search)
cv_results = pd.DataFrame(rf_search.cv_results_)
cv_results_sorted = cv_results.sort_values('mean_test_score', ascending=False)

print(f"📊 Available models from RandomizedSearchCV: {len(cv_results)}")
print(f"📊 Top 3 CV scores: {cv_results_sorted['mean_test_score'].head(3).values}")

# Erstelle die Top 3 RandomForest Modelle (ohne Pipeline)
top_3_models = []
for i in range(3):
    params = cv_results_sorted.iloc[i]['params']
    
    # Erstelle RandomForest mit den Top-Parametern
    rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
    rf_model.set_params(**params)
    
    # Trainiere das Modell
    rf_model.fit(X_train, y_train)
    
    top_3_models.append((f'rf_model_{i+1}', rf_model))
    
    print(f"✅ Model {i+1} trained with CV score: {cv_results_sorted.iloc[i]['mean_test_score']:.1f}")

print(f"\n🏗️ Creating Stacking Ensemble with {len(top_3_models)} base models...")

# Stacking Classifier mit LogisticRegression als Meta-Learner
# TimeSeriesSplit funktioniert nicht mit StackingClassifier, nutze Standard KFold
from sklearn.model_selection import KFold

stacking_classifier = StackingClassifier(
    estimators=top_3_models,
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=KFold(n_splits=5, shuffle=True, random_state=42),  # Standard CV für Stacking
    n_jobs=-1,
    verbose=True
)

print("🚀 Training Stacking Ensemble...")
stacking_classifier.fit(X_train, y_train)

# Stacking Predictions
stacking_predictions = stacking_classifier.predict(X_test)
stacking_score = kicktipp_score(y_test, stacking_predictions)

print(f"\n🎯 Stacking Ensemble Test Score: {stacking_score} points")

# Vergleiche auch die individuellen Modelle
print(f"\n📊 Individual model performance on test set:")
for i, (name, model) in enumerate(top_3_models):
    individual_predictions = model.predict(X_test)
    individual_score = kicktipp_score(y_test, individual_predictions)
    print(f"  {name}: {individual_score} points")


🏗️ Building Stacking Ensemble from Top 3 RandomForest models...
📊 Available models from RandomizedSearchCV: 150
📊 Top 3 CV scores: [343.4 340.8 340. ]
✅ Model 1 trained with CV score: 343.4
✅ Model 2 trained with CV score: 340.8
✅ Model 3 trained with CV score: 340.0

🏗️ Creating Stacking Ensemble with 3 base models...
🚀 Training Stacking Ensemble...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
/Users/maxlautenbach/PycharmProjects/football_match_prediction/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:1380: RuntimeWarning: Number of classes in training fold (47) does not match total number of classes (49). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
/Users/maxlautenbach/PycharmProjects/football_match_prediction/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:1380: RuntimeWarning: Number of classes in training fold (48) does not match total number of classes (49). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in prope


🎯 Stacking Ensemble Test Score: 315 points

📊 Individual model performance on test set:
  rf_model_1: 326 points
  rf_model_2: 324 points
  rf_model_3: 329 points


In [53]:
# Final Comparison: Original RF vs Stacking (Clean Version)
print("=" * 60)
print("🏆 FINAL MODEL COMPARISON")
print("=" * 60)
print(f"📊 Baseline Score:        {baseline_score} points")
print(f"🌳 Original RandomForest: {final_score} points")
print(f"🏗️ Stacking Ensemble:     {stacking_score} points")
print("=" * 60)

original_improvement = final_score - baseline_score
stacking_improvement = stacking_score - baseline_score
stacking_vs_original = stacking_score - final_score

print(f"📈 Original RF improvement:  +{original_improvement} points")
print(f"📈 Stacking improvement:     +{stacking_improvement} points")
print(f"🏗️ Stacking vs Original:     {stacking_vs_original:+} points")

if stacking_score > final_score:
    print(f"\n✅ STACKING WINS! Ensemble improved by {stacking_vs_original} points")
    best_final_model = stacking_classifier
    best_final_score = stacking_score
    best_approach = "Stacking"
else:
    print(f"\n🤔 Original RF still better by {-stacking_vs_original} points")
    best_final_model = best_model
    best_final_score = final_score
    best_approach = "Original RF"

print(f"\n🏆 WINNER: {best_approach} with {best_final_score} points")

# Komplexität vs Performance
print(f"\n💡 COMPLEXITY vs PERFORMANCE:")
print(f"  🌳 Original RF: {final_score:3.0f} points (1 model)")
print(f"  🏗️ Stacking:    {stacking_score:3.0f} points (3 models + meta-learner)")

if stacking_vs_original >= 3:
    print(f"\n🏆 RECOMMENDATION: Use Stacking Ensemble!")
    print(f"   ✅ Significant improvement: +{stacking_vs_original} points")
else:
    print(f"\n🏆 RECOMMENDATION: Use Original RandomForest!")
    print(f"   ✅ Simpler model with comparable performance")


🏆 FINAL MODEL COMPARISON
📊 Baseline Score:        301 points
🌳 Original RandomForest: 326 points
🏗️ Stacking Ensemble:     315 points
📈 Original RF improvement:  +25 points
📈 Stacking improvement:     +14 points
🏗️ Stacking vs Original:     -11 points

🤔 Original RF still better by 11 points

🏆 WINNER: Original RF with 326 points

💡 COMPLEXITY vs PERFORMANCE:
  🌳 Original RF: 326 points (1 model)
  🏗️ Stacking:    315 points (3 models + meta-learner)

🏆 RECOMMENDATION: Use Original RandomForest!
   ✅ Simpler model with comparable performance


In [54]:
# Save Best Model and Encoder (Clean Version)
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Determine which model performed better
if stacking_score > final_score:
    model_to_save = stacking_classifier
    score_to_save = stacking_score
    approach_name = "Stacking"
    model_type = "StackingClassifier with 3 RandomForest base models"
else:
    model_to_save = best_model
    score_to_save = final_score
    approach_name = "Original RF"
    model_type = "RandomForestClassifier"

model_filename = f"classifier_{timestamp}.pck"
pickle.dump(model_to_save, open(model_filename, "wb"))

# Save encoder
encoder_filename = "encoder.pck"
pickle.dump(enc, open(encoder_filename, "wb"))

print(f"💾 Best model ({approach_name}) saved as: {model_filename}")
print(f"💾 Model type: {model_type}")
print(f"💾 Encoder saved as: {encoder_filename}")
print(f"💾 Final score: {score_to_save} points")
print(f"💾 Improvement over baseline: +{score_to_save - baseline_score} points")


💾 Best model (Original RF) saved as: classifier_20250928_130858.pck
💾 Model type: RandomForestClassifier
💾 Encoder saved as: encoder.pck
💾 Final score: 326 points
💾 Improvement over baseline: +25 points


In [55]:
# Stacking Ensemble: Top 3 RandomForest Models
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd

print("🏗️ Building Stacking Ensemble from Top 3 RandomForest models...")

# Extrahiere Top 3 Modelle aus der RandomizedSearchCV
cv_results = pd.DataFrame(pipeline_search.cv_results_)
cv_results_sorted = cv_results.sort_values('mean_test_score', ascending=False)

print(f"📊 Available models from RandomizedSearchCV: {len(cv_results)}")
print(f"📊 Top 3 CV scores: {cv_results_sorted['mean_test_score'].head(3).values}")

# Erstelle die Top 3 RandomForest Modelle
top_3_models = []
for i in range(3):
    params = cv_results_sorted.iloc[i]['params']
    
    # Erstelle Pipeline mit gleicher Struktur
    model_pipeline = Pipeline([
        ('feature_engineering', HomeAwayDifferenceTransformer(verbose=False)),
        ('rf', RandomForestClassifier(random_state=42, n_jobs=-1))
    ])
    
    # Setze die Parameter
    model_pipeline.set_params(**params)
    
    # Trainiere das Modell
    model_pipeline.fit(X_train, y_train)
    
    top_3_models.append((f'rf_model_{i+1}', model_pipeline))
    
    print(f"✅ Model {i+1} trained with CV score: {cv_results_sorted.iloc[i]['mean_test_score']:.1f}")

print(f"\n🏗️ Creating Stacking Ensemble with {len(top_3_models)} base models...")

# Stacking Classifier mit LogisticRegression als Meta-Learner
stacking_classifier = StackingClassifier(
    estimators=top_3_models,
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=tscv,  # Gleiche CV-Strategie
    n_jobs=-1,
    verbose=True
)

print("🚀 Training Stacking Ensemble...")
stacking_classifier.fit(X_train, y_train)

# Stacking Predictions
stacking_predictions = stacking_classifier.predict(X_test)
stacking_score = kicktipp_score(y_test, stacking_predictions)

print(f"\n🎯 Stacking Ensemble Test Score: {stacking_score} points")

# Vergleiche auch die individuellen Modelle
print(f"\n📊 Individual model performance on test set:")
for i, (name, model) in enumerate(top_3_models):
    individual_predictions = model.predict(X_test)
    individual_score = kicktipp_score(y_test, individual_predictions)
    print(f"  {name}: {individual_score} points")


🏗️ Building Stacking Ensemble from Top 3 RandomForest models...
📊 Available models from RandomizedSearchCV: 150
📊 Top 3 CV scores: [346.8 345.8 345.4]
✅ Model 1 trained with CV score: 346.8
✅ Model 2 trained with CV score: 345.8
✅ Model 3 trained with CV score: 345.4

🏗️ Creating Stacking Ensemble with 3 base models...
🚀 Training Stacking Ensemble...


ValueError: cross_val_predict only works for partitions

In [ ]:
# Final Model Comparison: All Approaches
print("=" * 70)
print("🏆 FINAL MODEL COMPARISON - ALL APPROACHES")
print("=" * 70)
print(f"📊 Baseline Score:              {baseline_score} points")
print(f"🌳 Original RandomForest:       {final_score} points")
print(f"🔄 Pipeline (Feature Eng.):     {pipeline_final_score} points")
print(f"🏗️ Stacking Ensemble:           {stacking_score} points")
print("=" * 70)

# Alle Verbesserungen berechnen
all_scores = {
    'Baseline': baseline_score,
    'Original RF': final_score,
    'Pipeline RF': pipeline_final_score,
    'Stacking': stacking_score
}

improvements = {name: score - baseline_score for name, score in all_scores.items() if name != 'Baseline'}

print("📈 IMPROVEMENTS over baseline:")
for approach, improvement in improvements.items():
    print(f"  {approach:15}: +{improvement:4.0f} points")

# Bester Ansatz
best_approach_name = max(all_scores, key=all_scores.get)
best_score_final = all_scores[best_approach_name]

print(f"\n🏆 OVERALL WINNER: {best_approach_name} with {best_score_final} points!")

# Modell-Komplexität vs Performance
print(f"\n💡 COMPLEXITY vs PERFORMANCE ANALYSIS:")
print(f"  📊 Baseline:        {baseline_score:3.0f} points (Trivial)")
print(f"  🌳 Original RF:     {final_score:3.0f} points (Medium complexity)")
print(f"  🔄 Pipeline RF:     {pipeline_final_score:3.0f} points (Medium+ complexity)")  
print(f"  🏗️ Stacking:        {stacking_score:3.0f} points (High complexity)")

# Finale Empfehlung
stacking_improvement = stacking_score - max(final_score, pipeline_final_score)
if stacking_improvement >= 5:
    print(f"\n🏆 RECOMMENDATION: Stacking Ensemble!")
    print(f"   ✅ Best performance: {stacking_score} points")
    print(f"   ✅ Significant improvement: +{stacking_improvement} points")
    final_best_model = stacking_classifier
elif pipeline_final_score > final_score:
    print(f"\n🏆 RECOMMENDATION: Pipeline with Feature Engineering!")
    print(f"   ✅ Good performance with reasonable complexity")
    print(f"   ✅ Feature engineering benefits: +{pipeline_final_score - final_score} points")
    final_best_model = pipeline_best_model
else:
    print(f"\n🏆 RECOMMENDATION: Original RandomForest!")
    print(f"   ✅ Simple and effective: {final_score} points")
    final_best_model = best_model

print(f"\n💾 Best model ready for saving: {type(final_best_model).__name__}")


In [ ]:
# Save Best Model and Encoder
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the best performing model
model_filename = f"classifier_{timestamp}.pck"
pickle.dump(stacking_classifier, open(model_filename, "wb"))

# Save encoder
encoder_filename = "encoder.pck"
pickle.dump(enc, open(encoder_filename, "wb"))

print(f"💾 Stacking model saved as: {model_filename}")
print(f"💾 Encoder saved as: {encoder_filename}")
print(f"💾 Final score: {stacking_score} points")
